In [1]:
import numpy as np
import pandas as pd

In [2]:
#data = open('kafka.txt', 'r').read()
data = open('compression_data.txt',encoding="utf8").read()
data=data.replace(' ', ' _ ')
data_list=data.split(' ')
#data="Neural networks operate on vectors (a vector is an array of float)."
chars = list(set(data))
data_word_len=len(data_list)
words= list(set(data_list))
data_size, vocab_size, word_size = len(data), len(chars), len(words)
print ('data has %d chars, %d unique %d words' % (data_size, vocab_size, word_size))

data has 812 chars, 38 unique 66 words


In [22]:
data_list
len(data_list)

197

In [4]:
words

['spark\nLights',
 'not',
 'curtains',
 'your',
 'for',
 'peep,\nFor',
 'this',
 'nothing',
 'a',
 'light,\nTwinkle,',
 'never',
 'eye\nTill',
 'know',
 'go,\nIf',
 'he',
 'my',
 'above',
 'are,\nTwinkle,',
 'twinkle,',
 'where',
 'night.\n\nThen',
 'bright',
 'through',
 'shines',
 'keep,\nAnd',
 'and',
 'blazing',
 '_',
 'tiny',
 'upon,\nThen',
 'so.\n\nIn',
 'high,\nLike',
 'I',
 'world',
 'diamond',
 'sky.\n\nAs',
 'could',
 'sun',
 'traveller',
 'in',
 'wonder',
 'dark\nThanks',
 'star.',
 'Twinkle,',
 'often',
 'dark',
 'what',
 'sky',
 'blue',
 'dark,\nThough',
 'the',
 'see',
 'to',
 'gone,\nWhen',
 'you',
 'is',
 'spark;\nHe',
 'did',
 'star,\nHow',
 'little',
 'are!\nUp',
 'show',
 'shut',
 'so',
 'twinkle',
 'sky.\n\nWhen']

In [5]:
word_to_ix={word:i for i,word in enumerate(words)}
ix_to_word={i:word for i,word in enumerate(words)}
print(word_to_ix)
print('\n')
print(ix_to_word)

{'spark\nLights': 0, 'not': 1, 'curtains': 2, 'your': 3, 'for': 4, 'peep,\nFor': 5, 'this': 6, 'a': 8, 'above': 16, 'go,\nIf': 13, 'my': 15, 'are,\nTwinkle,': 17, 'nothing': 7, 'twinkle,': 18, 'night.\n\nThen': 20, 'eye\nTill': 11, 'bright': 21, 'through': 22, 'where': 19, 'keep,\nAnd': 24, 'never': 10, 'blazing': 26, '_': 27, 'upon,\nThen': 29, 'know': 12, 'so': 63, 'high,\nLike': 31, 'I': 32, 'diamond': 34, 'see': 51, 'sky.\n\nAs': 35, 'could': 36, 'world': 33, 'sun': 37, 'traveller': 38, 'in': 39, 'wonder': 40, 'little': 59, 'dark\nThanks': 41, 'star.': 42, 'Twinkle,': 43, 'often': 44, 'dark': 45, 'what': 46, 'light,\nTwinkle,': 9, 'blue': 48, 'shines': 23, 'dark,\nThough': 49, 'the': 50, 'and': 25, 'to': 52, 'gone,\nWhen': 53, 'you': 54, 'is': 55, 'spark;\nHe': 56, 'did': 57, 'sky': 47, 'star,\nHow': 58, 'are!\nUp': 60, 'show': 61, 'shut': 62, 'tiny': 28, 'he': 14, 'twinkle': 64, 'so.\n\nIn': 30, 'sky.\n\nWhen': 65}


{0: 'spark\nLights', 1: 'not', 2: 'curtains', 3: 'your', 4: 'for

In [6]:
vector_for_word_twinkle = np.zeros((word_size, 1))
vector_for_word_twinkle[word_to_ix['twinkle']] = 1
print (vector_for_word_twinkle.ravel())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [7]:
#model parameters

hidden_size = 100
seq_length = 5
learning_rate = 1e-1

Wxh = np.random.randn(hidden_size, word_size) * 0.01 #input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 #input to hidden
Why = np.random.randn(word_size, hidden_size) * 0.01 #input to hidden
bh = np.zeros((hidden_size, 1))
by = np.zeros((word_size, 1))

In [8]:

def lossFun(inputs, targets, hprev):
    t=0
  #"""                                                                                                                                                                                         
  #inputs,targets are both list of integers.                                                                                                                                                   
  #hprev is Hx1 array of initial hidden state                                                                                                                                                  
  #returns the loss, gradients on model parameters, and last hidden state                                                                                                                      
  #"""
  #store our inputs, hidden states, outputs, and probability values
    xs, hs, ys, ps, = {}, {}, {}, {} #Empty dicts
        # Each of these are going to be SEQ_LENGTH(Here 25) long dicts i.e. 1 vector per time(seq) step
        # xs will store 1 hot encoded input characters for each of 25 time steps (26, 25 times)
        # hs will store hidden state outputs for 25 time steps (100, 25 times)) plus a -1 indexed initial state
        # to calculate the hidden state at t = 0
        # ys will store targets i.e. expected outputs for 25 times (26, 25 times), unnormalized probabs
        # ps will take the ys and convert them to normalized probab for chars
        # We could have used lists BUT we need an entry with -1 to calc the 0th hidden layer
        # -1 as  a list index would wrap around to the final element
    xs, hs, ys, ps = {}, {}, {}, {}
      #init with previous hidden state
        # Using "=" would create a reference, this creates a whole separate copy
        # We don't want hs[-1] to automatically change if hprev is changed
    hs[-1] = np.copy(hprev)
    #init loss as 0
    loss = 0
  # forward pass                                                                                                                                                                              
    for t in range(len(inputs)):
        xs[t] = np.zeros((word_size,1)) # encode in 1-of-k representation (we place a 0 vector as the t-th input)                                                                                                                     
        xs[t][inputs[t]] = 1 # Inside that t-th input we use the integer in "inputs" list to  set the correct
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state                                                                                                            
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars                                                                                                           
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars                                                                                                              
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)                                                                                                                       
  # backward pass: compute gradients going backwards    
  #initalize vectors for gradient values for each set of weights 
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        #output probabilities
        dy = np.copy(ps[t])
        #derive our first gradient
        dy[targets[t]] -= 1 # backprop into y  
        #compute output gradient -  output times hidden states transpose
        #When we apply the transpose weight matrix,  
        #we can think intuitively of this as moving the error backward
        #through the network, giving us some sort of measure of the error 
        #at the output of the lth layer. 
        #output gradient
        dWhy += np.dot(dy, hs[t].T)
        #derivative of output bias
        dby += dy
        #backpropagate!
        dh = np.dot(Why.T, dy) + dhnext # backprop into h                                                                                                                                         
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
        dbh += dhraw #derivative of hidden bias
        dWxh += np.dot(dhraw, xs[t].T) #derivative of input to hidden layer weight
        dWhh += np.dot(dhraw, hs[t-1].T) #derivative of hidden layer to hidden layer weight
        dhnext = np.dot(Whh.T, dhraw) 
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]
    

In [9]:
#prediction, one full forward pass
def sample(h, seed_ix, n):
    """                                                                                                                                                                                         
    sample a sequence of integers from the model                                                                                                                                                
    h is memory state, seed_ix is seed letter for first time step   
    n is how many characters to predict
    """
    #create vector
    x = np.zeros((word_size, 1))
    #customize it for our seed char
    x[seed_ix] = 1
    #list to store generated chars
    ixes = []
    #for as many characters as we want to generate
    for t in range(n):
        #a hidden state at a given time step is a function 
        #of the input at the same time step modified by a weight matrix 
        #added to the hidden state of the previous time step 
        #multiplied by its own hidden state to hidden state matrix.
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        #compute output (unnormalised)
        y = np.dot(Why, h) + by
        ## probabilities for next chars
        p = np.exp(y) / np.sum(np.exp(y))
        #pick one with the highest probability 
        ix = np.random.choice(range(word_size), p=p.ravel())
        #create a vector
        x = np.zeros((word_size, 1))
        #customize it for the predicted char
        x[ix] = 1
        #add it to the list
        ixes.append(ix)

    txt = ''.join(ix_to_word[ix] for ix in ixes)
    print ('----\n %s \n----' % (txt, ))
hprev = np.zeros((hidden_size,1)) # reset RNN memory  
    #predict the 200 next characters given 'a'
sample(hprev,word_to_ix['a'],len(data_list))

----
 are,
Twinkle,curtainsstar.twinklenothingare,
Twinkle,keep,
Andaboveforcouldthroughworlddark,
ThoughtheIyoutinythesky.

Asforwhatkeep,
Andare!
Upwhereskyspark;
Hebluewonderworldnothingcoulddiamondshinesstar.go,
Ifdid_couldwhatbluetwinklestar,
Howinblazingtwinklenevershinesthisdiamondyourdarktheworldtwinkle,worldso.

Inheindarkwonderknowdark
ThanksyourtothewhatsunthroughnotTwinkle,aboveshutanddark
Thanksare!
Upsky.

Aswhereanotwheresoforbluebluethetwinklesoneverpeep,
ForcurtainsshinesTwinkle,curtainsTwinkle,are!
Upupon,
Thenshutspark;
Henotlight,
Twinkle,andcurtainshigh,
Likeisseeshutnight.

Thenblueoftenabovedidknowindark,
ThoughdarkdarktothisIwherewheregone,
Whencurtainshigh,
Likeyouthisare!
Upupon,
Thencouldstar,
Howthistravellerhigh,
Likemyknowlittleshowwhatlight,
Twinkle,high,
Likenotpeep,
Forstar.youneverthroughdark
Thanksshowtwinkle,throughtinywherethisthisnight.

Thenbrightspark
Lightssky.

Whentwinkleaboveknowyoukeep,
Andmyayoulight,
Twinkle,night.

Thenupon,
ThensunTwinkl

In [10]:
p=0
inputs = [word_to_ix[ch] for ch in data_list[p:p+seq_length]]
print ("inputs", inputs)
print (data_list[p:p+seq_length])
targets = [word_to_ix[ch] for ch in data_list[p+1:p+seq_length+1]]
print ("targets", targets)
print (data_list[p+1:p+seq_length+1])

inputs [43, 27, 18, 27, 59]
['Twinkle,', '_', 'twinkle,', '_', 'little']
targets [27, 18, 27, 59, 27]
['_', 'twinkle,', '_', 'little', '_']


In [11]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/word_size)*seq_length # loss at iteration 0                                                                                                                        
while n<=1000*100:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  # check "How to feed the loss function to see how this part works
    if p+seq_length+1 >= len(data_list) or n == 0:
        hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
        p = 0 # go from start of data                                                                                                                                                             
    inputs = [word_to_ix[ch] for ch in data_list[p:p+seq_length]]
    targets = [word_to_ix[ch] for ch in data_list[p+1:p+seq_length+1]]

  # forward seq_length characters through the net and fetch gradient                                                                                                                          
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

  # sample from the model now and then                                                                                                                                                        
    if n % 1000 == 0:
        print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
        sample(hprev, inputs[0], len(data_list))

  # perform parameter update with Adagrad                                                                                                                                                     
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],[dWxh, dWhh, dWhy, dbh, dby],[mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

    p += seq_length # move data pointer                                                                                                                                                         
    n += 1 # iteration counter
#sample(hprev, inputs[0], len(data_list))

iter 0, loss: 20.948274
----
 Twinkle,isblazingtonotdiddark,
ThoughskyTwinkle,Idark
Thanksabovewonderabovewonderdark,
Thoughskytwinkle,throughcouldthesundark
Thanksthistinylittleso.

Inknowwherewheresky.

Asthroughoftenneverstar,
HowI_dark
Thankskeep,
Andeye
Tilldark
Thankskeep,
Andso.

Inwonderso.

Inpeep,
ForTwinkle,curtainsyourdark
ThanksTwinkle,skyspark
Lightscurtainsyourdidlittleupon,
Thenaboveare,
Twinkle,so.

Inthego,
Iftinyskynothingnight.

Thennotbluewonderstar.dark_thisbrightdark,
Thoughanddark
Thanksknowyourpeep,
Forgone,
Whenforthetinyspark
Lightsisgone,
Whendiamonddark
Thankswhereso.

Inwhatknowcouldskynevertwinklegone,
Whenknownevercouldskystar.keep,
Andtinydark
Thanksnight.

ThenoftenIlittlesky.

Assky.

Asthisspark;
Heaare,
Twinkle,bluenight.

Thendark,
Thoughtwinklesunshuttodidabovewondershutspark
Lightseye
Tilldiamondworldforneverworldlittlegone,
Whenhetravellerthishesky.

Whendark,
Thoughyougo,
Ifshowsky.

Whenseeabovediamondshutseeare!
Updidoftendark,
Thoughdid_sost

iter 12000, loss: 0.139942
----
 _above_you_world_so_for_a_diamond_in_the_sky.

When_this_blazing_sun_is_gone,
When_he_nothing_shines_upon,
Then_so.

In_you_are,
Twinkle,_diamond_you_are!
Up_above_the_world_so_sky_he_keep,
And_often_through_the_night.

Then_the_traveller_in_the_dark,
Though_I_know_not_what_you_are,
Twinkle,_twinkle,_little___twinkle,_traveller_you_keep,
And_go,
If_the_dark
Thanks_you_sky.

As_your_bright_light,
Twinkle,_twinkle,_in_the_dark_blue_sky_you_are,
Twinkle,_twinkle,_little_show_your_eye
Till_the_night.

Then_the_traveller_in_the_sky.

When_this_blazing_sun_is_peep,
For_you_keep,
And_often_through_the_you_show_your_little_light,
Twinkle,_twinkle,_ 
----
iter 13000, loss: 0.088836
----
 _could_you_keep,
And_night.

Then_the_little_so_is_you_a_nothing_shines_upon,
Then_in_the_dark
Thanks_you_for_your_tiny_spark;
He_could_not_see_where_to_go,
If_you_did_not_twinkle_so.

In_the_dark_blue_sky_you_keep,
And_often_through_my_curtains_peep,
For_you_never_shut_your_eye

iter 25000, loss: 0.018004
----
 _what_you_show_your_little_light,
Twinkle,_twinkle,_through_the_night.

Then_the_traveller_in_the_dark,
Though_I_know_not_what_you_are,
Twinkle,_twinkle,_little_star,
How_I_wonder_what_you_keep,
And_above_the_world_so_high,
Like_a_diamond_in_the_sky.

When_this_blazing_sun_is_gone,
When_he_nothing_shines_upon,
Then_you_show_your_little_light,
Twinkle,_twinkle,_through_the_night.

Then_the_traveller_in_the_dark
Thanks_you_for_your_tiny_spark;
He_could_not_see_where_to_go,
If_you_did_not_twinkle_so.

In_the_dark_blue_sky_you_keep,
And_often_through_my_curtains_peep,
For_you_never_shut_your_eye
Till_the_sun_is_in_ 
----
iter 26000, loss: 0.016758
----
 peep,
For_you_never_shut_your_eye
Till_the_sun_is_in_the_sky.

As_your_bright_and_tiny_spark
Lights_the_traveller_in_the_dark,
Though_I_know_not_what_you_are,
Twinkle,_twinkle,_little_sun_I_twinkle_twinkle_so.

In_the_traveller_in_the_dark
Thanks_you_for_your_tiny_spark;
He_could_not_see_where_to_go,
If_you_

iter 38000, loss: 0.009590
----
 traveller_in_the_sky.

As_your_know_not_my_curtains_peep,
For_you_never_shut_your_eye
Till_the_sun_is_in_the_sky.

As_your_bright_and_tiny_spark
Lights_the_traveller_in_the_dark,
Though_I_know_not_what_you_are,
Twinkle,_twinkle,_little_little_tiny_not_you_did_sky.

As_the_sky.

When_you_for_your_eye
Till_the_night.

Then_the_world_so_the_a_diamond_in_the_what_show_above_the_dark,
Though_in_the_dark
Thanks_you_for_your_tiny_spark;
He_could_not_see_where_to_go,
If_you_did_not_twinkle_so.

In_the_dark_blue_sky_you_keep,
And_often_through_my_curtains_peep,
For_you_never_shut 
----
iter 39000, loss: 0.009124
----
 star,
How_I_wonder_what_you_are!
Up_above_the_world_so_high,
Like_a_diamond_in_the_sky.

When_this_blazing_sun_is_gone,
When_he_nothing_shines_upon,
Then_you_the_sun_is_gone,
When_keep,
And_often_spark;
He_could_not_see_where_to_go,
If_sun_to_go,
If_peep,
For_you_never_shut_your_eye
Till_the_sun_is_in_the_sky.

As_your_bright_and_tiny_spark
Lights_

iter 51000, loss: 0.006774
----
 _shut_your_eye
Till_the_sun_is_in_the_world_so_high,
Like_a_diamond_in_the_sky.

When_this_blazing_sun_is_gone,
When_he_nothing_shines_upon,
Then_you_show_your_little_light,
Twinkle,_twinkle,_through_the_night.

Then_the_traveller_in_the_dark
Thanks_you_for_your_tiny_spark;
He_could_not_see_where_to_go,
If_you_did_not_twinkle_so.

In_the_dark_blue_sky_you_keep,
And_often_through_my_curtains_peep,
For_you_never_shut_your_eye
Till_the_sun_is_in_the_sky.

As_your_bright_and_tiny_spark
Lights_the_traveller_in_the_dark,
Though_I_know_not_what_you_are,
Twinkle,_twinkle,_little_twinkle,_twinkle,_star,
How_ 
----
iter 52000, loss: 0.006556
----
 _night.

Then_the_not_sun_is_gone,
When_he_nothing_shines_upon,
Then_you_show_your_little_light,
Twinkle,_twinkle,_through_the_traveller_through_the_night.

Then_the_traveller_in_the_dark,
Though_often_shines_my_did_not_sun_not_go,
If_the_dark_blue_sky_you_keep,
And_often_through_my_curtains_peep,
For_you_never_shut_you

iter 64000, loss: 0.008134
----
 _what_you_are!
Up_a_diamond_in_the_sky.

When_this_blazing_sun_is_gone,
When_he_nothing_shines_upon,
Then_you_the_night.

Then_the_traveller_in_the_dark
Thanks_you_for_your_tiny_spark;
He_could_not_see_where_to_go,
If_you_did_not_twinkle_so.

In_the_dark_blue_sky_you_keep,
And_often_through_my_curtains_peep,
For_you_never_shut_your_eye
Till_the_sun_is_in_the_sky.

As_your_bright_and_tiny_spark
Lights_the_traveller_in_the_dark,
Though_I_know_not_what_you_are,
Twinkle,_twinkle,_little_are,
Twinkle,_twinkle,_little_sky.

When_high,
Like_a_upon,
Then_world_he_high,
Likeare,
Twinkle,a_diamond_in_the_dark
Thanks_you_for_ 
----
iter 65000, loss: 0.007301
----
 peep,
Forpeep,
Foryou_never_shutbluewonderskyworldhigh,
Like_a_diamond_in_the_sky.

As_your_bright_and_tiny_spark
Lights_the_traveller_in_the_dark,
Though_I_know_not_what_you_are,
Twinkle,_twinkle,_little_light,
Twinkle,_you_show_your_tiny_you_light,
Twinkle,_to_the_keep,
And_did_know_not_what_you_are,
T

iter 77000, loss: 0.004449
----
 traveller_in_the_dark
Thanks_you_for_your_tiny_spark;
He_could_not_see_where_to_go,
If_you_did_not_twinkle_so.

In_the_dark_blue_sky_you_keep,
And_often_through_my_curtains_peep,
For_you_never_shut_your_eye
Till_the_sun_is_in_the_sky.

As_your_bright_and_tiny_spark
Lights_the_traveller_in_the_dark,
Though_I_know_not_what_you_are,
Twinkle,_twinkle,_little_tiny_spark
Lights_know_not_twinkle_so.

In_traveller_you_you_show_your_eye
Till_my_curtains_you_show_your_little_dark_blue_above_the_world_so_he_nothing_shines_upon,
Then_you_show_your_little_light,
Twinkle,_twinkle,_through_the_night.

Then_the_traveller 
----
iter 78000, loss: 0.004464
----
 star,
How_I_wonder_what_you_are!
Up_above_the_world_so_high,
Like_a_diamond_in_the_sky.

When_this_blazing_sun_is_gone,
When_he_nothing_shines_upon,
Then_you_show_your_little_light,
Twinkle,_twinkle,_through_the_night.

Then_the_traveller_in_the_dark
Thanks_you_for_your_tiny_spark;
He_could_not_see_where_to_go,
If

iter 90000, loss: 0.008490
----
 _shut_your_eye
Till_the_sun_is_gone,
When_he_nothing_shines_upon,
Then_you_show_your_little_light,
Twinkle,_twinkle,_through_the_night.

Then_the_traveller_in_the_dark
Thanks_you_for_your_tiny_spark;
He_could_not_see_where_to_go,
If_you_did_not_twinkle_so.

In_the_dark_blue_sky_you_keep,
And_often_through_my_curtains_peep,
For_you_never_shut_your_eye
Till_the_sun_is_gone,
When_he_nothing_shines_upon,
Then_you_show_curtains_the_traveller_in_the_dark,
Though_I_know_not_through_my_curtains_tiny_spark
Lights_the_traveller_in_the_dark,
Though_I_know_not_what_you_are,
Twinkle,_twinkle,_little_light,
Twinkle,_you_never_twinkle_ 
----
iter 91000, loss: 0.007103
----
 _night.

Then_the_not_in_the_dark
Thanks_you_for_your_tiny_spark;
He_could_not_see_where_to_go,
If_you_did_not_twinkle_so.

In_the_dark_blue_sky_you_keep,
And_often_through_my_curtains_peep,
For_you_never_shut_your_eye
Till_the_sun_is_gone,
When_he_nothing_shines_upon,
Then_you_show_your_little_lig

In [12]:
p=0
inputs = [word_to_ix[ch] for ch in data_list[p:p+seq_length]]
print(inputs)
print(data_list[p:p+seq_length])

[43, 27, 18, 27, 59]
['Twinkle,', '_', 'twinkle,', '_', 'little']


In [13]:
sample(hprev, inputs[0], len(data_list))

----
 high,
Like_a_diamond_in_the_sky.

When_this_blazing_sun_is_gone,
When_he_nothing_shines_upon,
Then_you_show_your_you_keep,
And_often_not_my_curtains_tiny_spark
Lights_the_traveller_in_the_dark
Thanks_you_for_your_tiny_spark;
He_could_not_see_where_to_go,
If_you_did_not_twinkle_so.

In_the_dark_blue_sky_you_keep,
And_often_through_my_curtains_peep,
For_you_never_shut_your_eye
Till_the_sun_is_in_the_sky.

As_your_bright_and_tiny_spark
Lights_the_traveller_in_the_dark,
Though_I_know_not_what_you_are,
Twinkle,_twinkle,_little_the_is_this_blazing_sun_is_gone,
When_he_nothing_shines_upon,
Then_you 
----


In [14]:
df = pd.DataFrame(index=np.arange(100), columns=np.arange(100))

In [15]:
newDF = pd.DataFrame() #creates a new dataframe that's empty
newDF = newDF.append(pd.DataFrame(Wxh), ignore_index = True) # ignoring index is optional
newDF = newDF.append(pd.DataFrame(Whh), ignore_index = True)
newDF = newDF.append(pd.DataFrame(Why), ignore_index = True)
newDF = newDF.append(pd.DataFrame(bh).T, ignore_index = True)
newDF = newDF.append(pd.DataFrame(by).T, ignore_index = True)
newDF = newDF.append(pd.DataFrame(np.array([hidden_size,word_size])), ignore_index = True)
# try printing some data from newDF
print (newDF.head()) #again optional 

         0         1         2         3         4         5         6   \
0  0.417679 -1.015089  0.818057  1.002949  0.306640  0.428009  0.062594   
1 -0.714056  0.358035  0.820323 -0.123045  0.020488  0.095145 -0.389269   
2 -0.085588  1.239997 -0.136508 -0.425842 -0.228900  1.068405 -0.190006   
3  0.140041 -1.138674  1.081985 -0.924346  1.013271  0.486936 -0.828598   
4 -0.959936  0.106034 -0.113857  0.756349  0.051722 -0.079778 -0.335104   

         7         8         9  ...  90  91  92  93  94  95  96  97  98  99  
0 -1.373992 -0.439755  0.410488 ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
1 -0.956300  0.646310  0.378249 ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
2 -0.391968  1.036248 -2.419695 ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
3 -1.031534 -0.603040  0.147203 ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
4 -0.159425 -0.204105 -0.216692 ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  

[5 rows x 100 columns]


In [16]:
newDF

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.417679,-1.015089,0.818057,1.002949,0.306640,0.428009,0.062594,-1.373992,-0.439755,0.410488,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.714056,0.358035,0.820323,-0.123045,0.020488,0.095145,-0.389269,-0.956300,0.646310,0.378249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.085588,1.239997,-0.136508,-0.425842,-0.228900,1.068405,-0.190006,-0.391968,1.036248,-2.419695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.140041,-1.138674,1.081985,-0.924346,1.013271,0.486936,-0.828598,-1.031534,-0.603040,0.147203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.959936,0.106034,-0.113857,0.756349,0.051722,-0.079778,-0.335104,-0.159425,-0.204105,-0.216692,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.107003,0.351585,0.271831,0.977986,-0.925715,0.404494,1.207578,-1.839378,0.615618,0.168656,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.034880,0.818088,0.015303,-0.095977,-0.266912,0.482419,0.264478,-0.173647,-0.017002,0.665999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-0.187916,0.456755,0.019738,-0.322439,0.503154,-0.854010,0.528771,-0.148008,-0.546864,-0.022237,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.187544,0.066815,-0.894438,-1.958203,0.749819,-0.246711,-0.391201,0.799054,-0.374699,-0.142055,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.523695,-0.293527,-0.207621,0.575731,-0.110432,-0.048887,-0.616045,-0.397114,-0.086628,-0.670645,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
newDF.to_csv('out.csv')

In [18]:
df=pd.read_csv('out.csv')
header=df.dtypes.index
header=header.tolist()
df.drop(header[0], axis=1, inplace=True)

In [19]:
Wxh_read=np.array(df.iloc[0:hidden_size,0:word_size])
Whh_read=np.array(df.iloc[hidden_size:hidden_size*2,0:hidden_size])
Why_read=np.array(df.iloc[hidden_size*2:hidden_size*2+word_size,0:hidden_size])

In [20]:
bh_read=np.array(df[hidden_size*2+word_size:hidden_size*2+word_size+1].T)
by_read=np.array(df.iloc[hidden_size*2+word_size+1:hidden_size*2+word_size+2, 0:word_size].T)

In [21]:
def sample(h, seed_ix, n):
    """                                                                                                                                                                                         
    sample a sequence of integers from the model                                                                                                                                                
    h is memory state, seed_ix is seed letter for first time step   
    n is how many characters to predict
    """
    #create vector
    x = np.zeros((word_size, 1))
    #customize it for our seed char
    x[seed_ix] = 1
    #list to store generated chars
    ixes = []
    #for as many characters as we want to generate
    for t in range(n):
        #a hidden state at a given time step is a function 
        #of the input at the same time step modified by a weight matrix 
        #added to the hidden state of the previous time step 
        #multiplied by its own hidden state to hidden state matrix.
        h = np.tanh(np.dot(Wxh_read, x) + np.dot(Whh_read, h) + bh_read)
        #compute output (unnormalised)
        y = np.dot(Why_read, h) + by_read
        ## probabilities for next chars
        p = np.exp(y) / np.sum(np.exp(y))
        #pick one with the highest probability 
        ix = np.random.choice(range(word_size), p=p.ravel())
        #create a vector
        x = np.zeros((word_size, 1))
        #customize it for the predicted char
        x[ix] = 1
        #add it to the list
        ixes.append(ix)

    txt = ''.join(ix_to_word[ix] for ix in ixes)
    print ('----\n %s \n----' % (txt, ))
hprev = np.zeros((hidden_size,1)) # reset RNN memory  
    #predict the 200 next characters given 'a'
sample(hprev, inputs[0], len(data_list))

----
 _twinkle,_little_star,
How_I_wonder_what_you_are!
Up_above_the_world_so_high,
Like_a_diamond_in_the_sky.

When_this_blazing_sun_is_gone,
When_he_nothing_shines_upon,
Then_you_show_your_little_light,
Twinkle,_twinkle,_through_the_night.

Then_the_dark,
Though_in_the_dark,
Though_I_know_not_twinkle_so.

In_the_dark_blue_sky_you_keep,
And_often_through_my_curtains_peep,
For_you_never_shut_your_eye
Till_the_sun_is_in_the_sky.

As_your_bright_and_tiny_spark
Lights_the_traveller_in_the_dark,
Though_I_know_not_what_you_are,
Twinkle,_twinkle,_little_the_tiny_through_the_dark
Thanks_you_for_your_tiny_spark;
He_could_not_ 
----
